### Student Information
Name: 許昱呈

Student ID: 111164519

GitHub ID: a19990702

Kaggle name: BrianHSU8872

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__. 

---
## 1. Data Preparation

In [2]:
!python --version
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import gensim
import tensorflow
import keras
%matplotlib inline
print("gensim: " + gensim.__version__)
print("tensorflow: " + tensorflow.__version__)
print("keras: " + keras.__version__)

Python 3.6.10 :: Anaconda, Inc.


gensim: 4.2.0
tensorflow: 2.6.2
keras: 2.6.0


Using TensorFlow backend.


In [3]:
import pandas as pd
import json
### prepare the data
data_identification = pd.read_csv("dm2022-isa5810-lab2-homework/data_identification.csv")
emotion = pd.read_csv("dm2022-isa5810-lab2-homework/emotion.csv")
sampleSubmission = pd.read_csv("dm2022-isa5810-lab2-homework/sampleSubmission.csv")

In [4]:
file = open("dm2022-isa5810-lab2-homework/tweets_DM.json",'r',encoding='utf-8')
datas =[]
for line in file.readlines():
    data = json.loads(line)
    data['hashtags'] =  data['_source']['tweet']['hashtags']
    data['tweet_id'] = data['_source']['tweet']['tweet_id']
    data['text'] = data['_source']['tweet']['text']
    del data['_source']
    datas.append(data)
df = pd.DataFrame(datas)

In [5]:
df_data_identification = df.merge(data_identification, on = 'tweet_id', how = 'left')
# splite the data to two piles, data_x for training module, and put test_x data into module and get answer 
data_x = df_data_identification.loc[df_data_identification['identification'] == 'train']
test_x = df_data_identification.loc[df_data_identification['identification'] == 'test']
data_answer_y = data_x.merge(emotion, on = 'tweet_id', how = 'left' )['emotion']

In [6]:
'''
## check library
import gensim
import nltk 
## ignore warnings
import warnings
warnings.filterwarnings('ignore')

# # if you want to see the training messages, you can use it
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## the input type
data_x['text_tokenized'] = data_x['text'].apply(lambda x: nltk.word_tokenize(x))
'''

"\n## check library\nimport gensim\nimport nltk \n## ignore warnings\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# # if you want to see the training messages, you can use it\n# import logging\n# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)\n\n## the input type\ndata_x['text_tokenized'] = data_x['text'].apply(lambda x: nltk.word_tokenize(x))\n"

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
# build analyzers (bag-of-words)
# I try to use TFIDF into analyzers, and kill the stop words
BOW_vectorizer = TfidfVectorizer(stop_words='english')
# 1. Learn a vocabulary dictionary of all tokens in the raw documents.
BOW_vectorizer.fit(data_x['text'])

# 2. Transform documents to document-term matrix.
train_data_BOW_features = BOW_vectorizer.transform(data_x['text'])
test_data_BOW_features = BOW_vectorizer.transform(test_x['text'])

In [8]:
feature_names = BOW_vectorizer.get_feature_names()

In [9]:
import nltk

# build analyzers (bag-of-words)
#use TFIDF analyzers and decrease the feature to the 500
BOW_500 = TfidfVectorizer(max_features=500, tokenizer=nltk.word_tokenize,stop_words='english') 

# apply analyzer to training data
BOW_500.fit(data_x['text'])

train_data_BOW_features_500 = BOW_500.transform(data_x['text'])

## check dimension
train_data_BOW_features_500.shape

C:\Users\ferry\miniconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1455563, 500)

In [10]:
from sklearn.tree import DecisionTreeClassifier
#using decision tree to train the module
X_train = BOW_500.transform(data_x['text'])



In [11]:
# y_train is the x_train datasets answer
y_train = data_answer_y

X_test = BOW_500.transform(test_x['text'])

In [12]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=1)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
sampleSubmission['emotion'] = y_test_pred


In [13]:
sampleSubmission['emotion']

0                trust
1                trust
2              disgust
3         anticipation
4         anticipation
              ...     
411967    anticipation
411968    anticipation
411969    anticipation
411970         sadness
411971         sadness
Name: emotion, Length: 411972, dtype: object

In [14]:
sampleSubmission.to_csv("dm2022-isa5810-lab2-homework/sampleSubmission.csv",index=False)

## Third part
In the homework, 
I use the way in the lab2_master to use the BOW and train the module and get answer, 
I can get 18.55% accuracy
afterthat,
I use TFIDF to be the analyzer, get 20% accuracy
All of the training module use the decision Tree

Because of a lots of homework in November. I don't use a lots of time to find the way to get better answer
